In [1]:
!pip install yfinance pandas numpy scikit-learn matplotlib seaborn


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from datetime import datetime, timedelta

# Funktion zur Eingabe des Tickersymbols
def get_ticker():
    ticker = input("Bitte geben Sie das Tickersymbol der Aktie ein (Standard ist AAPL): ").strip().upper()
    if not ticker:
        ticker = 'AAPL'
    return ticker

# Funktion zur Eingabe und Vorhersage
def predict_for_date(prediction_date, data, model, features, verbose=False):
    # Berechnung des Start- und Enddatums für die letzten 10 Tage
    start_date = prediction_date - timedelta(days=10)
    end_date = prediction_date - timedelta(days=1)

    # Daten für den Vorhersagezeitraum extrahieren
    last_10_days = data.loc[start_date:end_date]
    if last_10_days.empty:
        print(f"Nicht genügend Daten für den Zeitraum von {start_date} bis {end_date}.")
        return

    if verbose:
        print(f"Analyse der letzten 10 Tage vom {start_date.strftime('%d.%m.%Y')} bis {end_date.strftime('%d.%m.%Y')}:")
        print(last_10_days[features])

    # Features für die Vorhersage berechnen
    last_10_days_features = last_10_days[features].tail(1)  # Nimm die letzten Zeilen, um die Struktur beizubehalten

    # Vorhersage treffen
    prediction = model.predict(last_10_days_features)
    
    # Preis für das Vorhersage-Datum abrufen
    if prediction_date in data.index:
        prediction_price = data.loc[prediction_date]['Adj Close']
        print(f"Preis für den {prediction_date.strftime('%d.%m.%Y')}: ${prediction_price:.2f}")
    else:
        print(f"Kein Preis verfügbar für das Datum {prediction_date.strftime('%d.%m.%Y')}.")
    
    # Preis für den nächsten Handelstag abrufen
    next_day = prediction_date + timedelta(days=1)
    
    if next_day in data.index:
        next_day_price = data.loc[next_day]['Adj Close']
        print(f"Preis für den {next_day.strftime('%d.%m.%Y')}: ${next_day_price:.2f}")
    else:
        print(f"Kein Preis verfügbar für den {next_day.strftime('%d.%m.%Y')}. Der Markt ist möglicherweise geschlossen.")
    
    print(f"Vorhersage für den {prediction_date.strftime('%d.%m.%Y')}: {'Kauf' if prediction[0] == 1 else 'Verkauf' if prediction[0] == -1 else 'Halten'}")

# Hauptprogramm
def main():
    ticker = get_ticker()
    start_date = '2010-01-01'
    end_date = '2023-08-21'
    
    # Daten herunterladen
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Berechnung der gleitenden Durchschnitte (Moving Averages)
    data['MA5'] = data['Adj Close'].rolling(window=5).mean()
    data['MA10'] = data['Adj Close'].rolling(window=10).mean()

    # Relative Strength Index (RSI) berechnen
    delta = data['Adj Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # Bollinger-Bänder berechnen
    data['MiddleBand'] = data['Adj Close'].rolling(window=20).mean()
    data['UpperBand'] = data['MiddleBand'] + 2 * data['Adj Close'].rolling(window=20).std()
    data['LowerBand'] = data['MiddleBand'] - 2 * data['Adj Close'].rolling(window=20).std()

    # Momentum berechnen
    data['Momentum'] = data['Adj Close'] - data['Adj Close'].shift(4)

    # Price Rate of Change (ROC)
    data['ROC'] = data['Adj Close'].pct_change(periods=10) * 100

    # Tagesveränderung des Preises in Prozent
    data['PriceChange'] = data['Adj Close'].pct_change() * 100

    # Volatilität über 10 Tage
    data['Volatility'] = data['Adj Close'].rolling(window=10).std()

    # Entfernen von Zeilen mit NaN-Werten
    data.dropna(inplace=True)
    print(data)
    # Training- und Testperioden definieren
    train_start = '2010-01-01'
    train_end = '2020-12-31'
    test_start = '2021-01-01'

    # Trainings- und Testdaten erstellen
    train_data = data.loc[train_start:train_end].copy()
    test_data = data.loc[test_start:].copy()

    # Zielvariable definieren
    threshold = 2  # Schwellenwert für Kauf- und Verkaufssignal (in Prozent)

    # Für Trainingsdaten
    train_data['Signal'] = 0
    train_data.loc[train_data['PriceChange'] > threshold, 'Signal'] = 1
    train_data.loc[train_data['PriceChange'] < -threshold, 'Signal'] = -1

    # Für Testdaten
    test_data['Signal'] = 0
    test_data.loc[test_data['PriceChange'] > threshold, 'Signal'] = 1
    test_data.loc[test_data['PriceChange'] < -threshold, 'Signal'] = -1

    # Features und Zielvariable definieren
    features = ['MA5', 'MA10', 'RSI', 'UpperBand', 'LowerBand', 'Momentum', 'ROC', 'Volatility']
    X_train = train_data[features]
    y_train = train_data['Signal']
    X_test = test_data[features]
    y_test = test_data['Signal']

    # Modell initialisieren
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Modell trainieren
    model.fit(X_train, y_train)

    # Vorhersage auf Testdaten
    y_pred = model.predict(X_test)

    # Ergebnisse anzeigen
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))

    # Benutzerabfrage für Vorhersage
    def get_prediction_date():
        while True:
            try:
                user_input = input("Bitte geben Sie das Datum für die Vorhersage im Format 'YYYY-MM-DD' ein: ")
                prediction_date = datetime.strptime(user_input, '%Y-%m-%d')
                return prediction_date
            except ValueError:
                print("Ungültiges Datum. Bitte geben Sie das Datum im Format 'YYYY-MM-DD' ein.")

    while True:
        verbose = input("Möchten Sie detaillierte Daten der letzten 10 Tage anzeigen? (ja/nein): ").strip().lower() == 'ja'
        prediction_date = get_prediction_date()
        predict_for_date(prediction_date, data, model, features, verbose)
        
        another_day = input("Möchten Sie ein weiteres Datum eingeben? (ja/nein): ").strip().lower()
        if another_day != 'ja':
            print("Programm beendet.")
            break

# Hauptprogramm starten
main()


Bitte geben Sie das Tickersymbol der Aktie ein (Standard ist AAPL):  


[*********************100%%**********************]  1 of 1 completed


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2010-02-01    6.870357    7.000000    6.832143    6.954643    5.873024   
2010-02-02    6.996786    7.011429    6.906429    6.995000    5.907104   
2010-02-03    6.970357    7.150000    6.943571    7.115357    6.008745   
2010-02-04    7.026071    7.084643    6.841786    6.858929    5.792195   
2010-02-05    6.879643    7.000000    6.816071    6.980714    5.895041   
...                ...         ...         ...         ...         ...   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546371   
2023-08-15  178.880005  179.479996  177.050003  177.449997  176.546600   
2023-08-16  177.130005  178.539993  176.500000  176.570007  175.671082   
2023-08-17  177.139999  177.509995  173.479996  174.000000  173.114151   
2023-08-18  172.300003  175.100006  171.960007  174.490005  173.601669   

               Volume         MA5    

Möchten Sie detaillierte Daten der letzten 10 Tage anzeigen? (ja/nein):  ja
Bitte geben Sie das Datum für die Vorhersage im Format 'YYYY-MM-DD' ein:  2022-06-01


Analyse der letzten 10 Tage vom 22.05.2022 bis 31.05.2022:
                   MA5        MA10        RSI   UpperBand   LowerBand  \
Date                                                                    
2022-05-23  139.875748  142.652084  37.313773  166.051214  132.885729   
2022-05-24  138.121652  141.254533  28.754571  165.738237  131.597695   
2022-05-25  138.062393  140.663911  34.213586  165.277209  130.496201   
2022-05-26  139.332535  140.784409  37.326315  163.270028  130.565583   
2022-05-27  141.712820  141.034290  47.755091  162.392736  130.674593   
2022-05-31  142.844693  141.360220  44.574173  161.350342  130.839125   

            Momentum       ROC  Volatility  
Date                                        
2022-05-23 -6.054398 -5.885846    5.232730  
2022-05-24 -0.454330 -9.157973    4.000378  
2022-05-25  3.130920 -4.081881    3.870097  
2022-05-26  6.113678  0.855790    3.893568  
2022-05-27  6.449493  1.719805    4.276630  
2022-05-31  8.375443  2.267416    4.61687